In [469]:
import gurobipy as gb 
from gurobipy import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [470]:
# Demand Projection Data

demand_projection_data = pd.read_excel('code_input_demand_projection.xlsx')
demand_projection = demand_projection_data["Demand"].tolist()
demand_years = demand_projection_data["Year"].tolist()

In [471]:
# Available Sources

sources = ['Hydro', 'Wind', 'Nuclear', 'Geothermal', 'Biomass', 'Solar']

emission = [0.028, 0.014, 0.008, 0.038, 0.23, 0.064]  #CO2 emissions per MWh

setup_cost = [5316, 1265, 6191, 2521, 4097, 1313] #$/kW

fixed_cost_kw = [29.86, 26.34, 95, 128.54, 125.72, 15.25] #$/kW/year  
fixed_cost_mwh = [x*1000/(24*30*12) for x in fixed_cost_kw] #$/MWh/year

var_cost_mwh_doc = [0, 0, 3, 1.16, 4.83, 0] #$/MWh/year

var_cost_mwh = []
for i in range(len(var_cost_mwh_doc)):
    var_cost_mwh.append(var_cost_mwh_doc[i] + fixed_cost_mwh[i])


supply_input = pd.read_excel('code_input_supply.xlsx')
supply_input['Capacity'] = supply_input['Capacity']*24*30*12
total_capacity_existing = sum(supply_input['Capacity'].tolist())

In [472]:
prob = gb.Model("Power Grid Optimization - Model 2")
prob.params.LogToConsole = 0

I = len(sources) #New power plants
J = len(demand_years) #Year of construction of new power plants
K = len(demand_years) #Year of Production

In [473]:
# Decision Variables

eh = prob.addVars(K, vtype=GRB.CONTINUOUS, name = [f"Electricity generated using Existing Hydro Plants in {k}" for k in demand_years])

x = prob.addVars(I, J, K, vtype=GRB.CONTINUOUS, name = [f"Qty of Electricity generated using {i} built in {j} in {k}" for i in sources for j in demand_years for k in demand_years])

c = prob.addVars(I, J, vtype=GRB.CONTINUOUS, name = [f"Capacity of Power Plant {i} built in {j}" for i in sources for j in demand_years])

e = prob.addVars(K, vtype=GRB.CONTINUOUS, name = [f"CO2 emission in {k}" for k in demand_years])


In [474]:
#Objective Function

#1. Minimize the total cost of yearly power generation

cost_existing = sum(eh[k]*var_cost_mwh[0] for k in range(K))
cost_new = sum(x[i,j,k]*(var_cost_mwh[i] + var_cost_mwh[i]) for i in range(I) for j in range(J) for k in range(K))

#2. Cost of setting up new plants

cost_setup = sum((setup_cost[i]*1000)*c[i,j]/(24*30*12) for i in range(I) for j in range(J))

total_cost_func = cost_existing + cost_new + cost_setup

#3. Minimize the total CO2 emission

total_emission = sum(e[k] for k in range(K))

prob.setObjectiveN(total_emission, index=0, priority=2)
prob.setObjectiveN(total_cost_func, index=1, priority=1)

prob.modelSense = GRB.MINIMIZE

In [475]:
#Constraints

constr_count = 0

#1. Power plants cannot run at more than 80% capacity
prob.addConstrs(x[i,j,k]<=0.8*c[i,j] for i in range(I) for j in range(J) for k in range(J))
prob.addConstrs(eh[k]<=0.8*total_capacity_existing for k in range(K))
constr_count += I*J*K + K

#2. Total Production each year must be atleast 35% higher than the demand in each month - to consider unexpected demand + transmission losses
prob.addConstrs(sum(x[i,j,k] for i in range(I) for j in range(J)) + eh[k] == 1.35*demand_projection[k] for k in range(K))
constr_count += K

#3. A power plant can only generate electricity after it is built

prob.addConstrs(x[i,j,k] <= c[i,j] for i in range(I) for j in range(J) for k in range(K) if j<=k)
prob.addConstrs(x[i,j,k] == 0 for i in range(I) for j in range(J) for k in range(K) if j>k)
constr_count += I*J*K + I*J*K

#4. Definition of the emission variable

prob.addConstrs(e[k] == sum(x[i,j,k]*emission[i] for i in range(I) for j in range(J)) + (eh[k]*emission[0]) for k in range(K))
constr_count += K

#5. Max investment that can be added in a year - assuming it is 4 billion dollars
max_inv_cost = 4*10**9
prob.addConstrs(sum((setup_cost[i]*1000)*c[i,j]/(24*30*12) for i in range(I))<= max_inv_cost for j in range(J))

#6. Maximum Demand that can be met with wind should be less than 15% of the total demand
prob.addConstrs(sum(x[1,j,k] for j in range(J)) <= 0.15*1.35*demand_projection[k] for k in range(K))
constr_count += K

#7. Atleast 70% of the total demand should be met with existing hydro plants
prob.addConstrs(eh[k]  >= 0.7*1.35*demand_projection[k] for k in range(K))
constr_count += K


print(f"{constr_count} constraints added to the model")

13257 constraints added to the model


In [476]:
prob.optimize()
if prob.status == GRB.OPTIMAL:
    print("Optimal Solution Found")
else:
    print("No Optimal Solution Found")

Optimal Solution Found


In [477]:
#Print the Reduction in CO2 Emission in 2050

print(f"Total Expected Reduction in CO2 emission in 2050: {abs(round((e[26].x-1.35*demand_projection[26]*emission[0])*100/(1.35*demand_projection[26]*emission[0]), 2))}%")

Total Expected Reduction in CO2 emission in 2050: 21.43%


In [478]:
# Construction Plan for New Power Plants

construction_plan = pd.DataFrame(columns = ['Year', 'Power Plant', 'Capacity (MW)'])

for i in range(I):
    for j in range(J):
        if c[i,j].x > 0:
            construction_plan = pd.concat([construction_plan, pd.DataFrame({'Year': demand_years[j], 'Power Plant': sources[i], 'Capacity (MW)': c[i,j].x}, index=[0])], ignore_index=True)

construction_plan['Capacity (MW)'] = np.ceil(construction_plan['Capacity (MW)']/(24*30*12))

construction_plan_df = pd.pivot_table(construction_plan, values='Capacity (MW)', index=['Power Plant'], columns='Year', aggfunc=np.sum, fill_value=0).reset_index()
construction_plan_df.rename(columns={'Power Plant':'Power Plant Capacity (in MW)'})

# Investment Required for New Power Plants

investment_plan = construction_plan.copy()
investment_plan['Investment Required'] = 0
investment_plan.loc[investment_plan['Power Plant'] == 'Hydro', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[0]*1000
investment_plan.loc[investment_plan['Power Plant'] == 'Wind', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[1]*1000
investment_plan.loc[investment_plan['Power Plant'] == 'Nuclear', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[2]*1000
investment_plan.loc[investment_plan['Power Plant'] == 'Geothermal', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[3]*1000
investment_plan.loc[investment_plan['Power Plant'] == 'Biomass', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[4]*1000
investment_plan.loc[investment_plan['Power Plant'] == 'Solar', 'Investment Required'] = investment_plan['Capacity (MW)']*setup_cost[5]*1000

investment_plan_df = pd.pivot_table(investment_plan, values='Investment Required', index=['Year'], aggfunc=np.sum, fill_value=0).reset_index()
investment_plan_df['Investment Required'] = investment_plan_df['Investment Required']/1000000
investment_plan_df.rename(columns={'Investment Required':'Investment Required (in Million $)'}, inplace=True)
total_investment_required = investment_plan_df['Investment Required (in Million $)'].sum()
investment_plan_df = investment_plan_df.append({'Year':'Total','Investment Required (in Million $)':total_investment_required}, ignore_index=True)

with pd.ExcelWriter('NewPowerPlant_ConstructionPlan.xlsx') as writer:
    investment_plan_df.to_excel(writer, sheet_name='Investment Required', index=False)
    construction_plan_df.to_excel(writer, sheet_name='Construction Plan', index=False)

In [479]:
# Power Generation from different sources

power_generation_plan = pd.DataFrame(columns = ['Year', 'Power Plant', 'Electricity Generated (MWh)'])

for i in range(I):
    for j in range(J):
        for k in range(K):
            if x[i,j,k].x > 0:
                power_generation_plan = pd.concat([power_generation_plan, pd.DataFrame({'Year': demand_years[k], 'Power Plant': sources[i], 'Electricity Generated (MWh)': x[i,j,k].x}, index=[0])], ignore_index=True)

for k in range(K):
    if eh[k].x > 0:
        power_generation_plan = pd.concat([power_generation_plan, pd.DataFrame({'Year': demand_years[k], 'Power Plant': 'Existing Hydro', 'Electricity Generated (MWh)': eh[k].x}, index=[0])], ignore_index=True)

power_generation_plan['Electricity Generated (MWh)'] = np.ceil(power_generation_plan['Electricity Generated (MWh)'])

power_generation_plan_df = pd.pivot_table(power_generation_plan, values='Electricity Generated (MWh)', index=['Power Plant'], columns=['Year'], aggfunc=np.sum, fill_value=0).reset_index()

power_generation_split_df = power_generation_plan_df.copy()
power_generation_split_df.loc[:, 2024:] = np.round((power_generation_plan_df.loc[:, 2024:] / power_generation_plan_df.loc[:, 2024:].sum(axis=0)) * 100,1)

with pd.ExcelWriter('PowerGeneration_Plan_2050.xlsx') as writer:
    power_generation_split_df.to_excel(writer, sheet_name='Percentage Distribution ', index=False)
    power_generation_plan_df.to_excel(writer, sheet_name='Power Generation', index=False)